In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
df=pd.read_excel(r"C:\Users\ultra_instinct\Downloads\Online Retail.xlsx",encoding='unicode_escape')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


### Basic information gathering

In [3]:
df.shape

(541909, 8)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [5]:
df.isnull()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...
541904,False,False,False,False,False,False,False,False
541905,False,False,False,False,False,False,False,False
541906,False,False,False,False,False,False,False,False
541907,False,False,False,False,False,False,False,False


In [6]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [7]:
df.nunique()

InvoiceNo      25900
StockCode       4070
Description     4223
Quantity         722
InvoiceDate    23260
UnitPrice       1630
CustomerID      4372
Country           38
dtype: int64

### Data Cleaning and Filling Missing Values

In [8]:
df[df['Quantity']<0].head()

#Checking which rows have quantity less than 0 because we dont need the rows where the quantity is less than 0

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom


In [9]:
df['InvoiceNo']=df['InvoiceNo'].astype('str') #converting the values in InvoiceNo to string

In [10]:
df = df[~df['InvoiceNo'].str.contains('C',na=False)] #returning all those rows which do not contain "C" in the InvoiceNo

In [11]:
df.shape

(532621, 8)

In [12]:
df["InvoiceNo"]

0         536365
1         536365
2         536365
3         536365
4         536365
           ...  
541904    581587
541905    581587
541906    581587
541907    581587
541908    581587
Name: InvoiceNo, Length: 532621, dtype: object

In [13]:
#Filling missing values 
df['CustomerID']=df['CustomerID'].fillna('00000')
df['Description']=df['Description'].fillna("Unknown")

In [14]:
df.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

### Feature Engineering

In [15]:
df['InvoiceDate']=pd.to_datetime(df['InvoiceDate'],errors='coerce') #three types of errors are there ignore,raise,coerce
df['Year']=df['InvoiceDate'].dt.year                                #coerce is when invalid parsing is set to NaN
df['Month']=df['InvoiceDate'].dt.month
df['Hour']=df['InvoiceDate'].dt.hour
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Year,Month,Hour
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,2010,12,8
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010,12,8
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,2010,12,8
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010,12,8
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010,12,8


In [16]:
df.shape

(532621, 11)

In [17]:
df['Month_Year']=df['InvoiceDate'].dt.to_period('M')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Year,Month,Hour,Month_Year
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,2010,12,8,2010-12
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010,12,8,2010-12
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,2010,12,8,2010-12
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010,12,8,2010-12
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010,12,8,2010-12


___Now we are finding out the income(revenues) generated by each product___

income = quantity * price of the product

In [18]:
df['Income']=round(df['Quantity']*df['UnitPrice'],2)
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Year,Month,Hour,Month_Year,Income
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,2010,12,8,2010-12,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010,12,8,2010-12,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,2010,12,8,2010-12,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010,12,8,2010-12,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010,12,8,2010-12,20.34


### Exploratory Data Analysis(EDA)

In [19]:
df.describe()

,Quantity,UnitPrice,Year,Month,Hour,Income
count,532621.000000,532621.000000,532621.000000,532621.000000,532621.000000,532621.000000
mean,10.239972,3.847621,2010.921608,7.557864,13.077451,19.985244
std,159.593551,41.758023,0.268787,3.508729,2.438535,270.574241
min,-9600.000000,-11062.060000,2010.000000,1.000000,6.000000,-11062.060000
25%,1.000000,1.250000,2011.000000,5.000000,11.000000,3.750000
50%,3.000000,2.080000,2011.000000,8.000000,13.000000,9.900000
75%,10.000000,4.130000,2011.000000,11.000000,15.000000,17.700000
max,80995.000000,13541.330000,2011.000000,12.000000,20.000000,168469.600000


In [20]:
df[df['Quantity']>100].head() #No of products whose quantity is greater than 100

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Year,Month,Hour,Month_Year,Income
96,536378,21212,PACK OF 72 RETROSPOT CAKE CASES,120,2010-12-01 09:37:00,0.42,14688,United Kingdom,2010,12,9,2010-12,50.40
178,536387,79321,CHILLI LIGHTS,192,2010-12-01 09:58:00,3.82,16029,United Kingdom,2010,12,9,2010-12,733.44
179,536387,22780,LIGHT GARLAND BUTTERFILES PINK,192,2010-12-01 09:58:00,3.37,16029,United Kingdom,2010,12,9,2010-12,647.04
180,536387,22779,WOODEN OWLS LIGHT GARLAND,192,2010-12-01 09:58:00,3.37,16029,United Kingdom,2010,12,9,2010-12,647.04
181,536387,22466,FAIRY TALE COTTAGE NIGHTLIGHT,432,2010-12-01 09:58:00,1.45,16029,United Kingdom,2010,12,9,2010-12,626.40


___Each product is assigned its unique StockCode hence for identifying the number of unique products we will have have to find the no. of unique StockCode !! Total no. of unique values can be found out by nunique()___

In [21]:
df['StockCode'].nunique()

4059

__No. of unique products ordered by customers:__

In [22]:
products_group=df[['CustomerID','StockCode']].groupby('CustomerID').count().sort_values('StockCode',axis=0,ascending=True)
products_group.head()

,StockCode
CustomerID,
12346.0,1
15668.0,1
15657.0,1
15562.0,1
15524.0,1


In [23]:
df['Country'].unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Finland',
       'Austria', 'Bahrain', 'Israel', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

In [24]:
df[df['Country']=='Poland'].head() #Analyzing data for poland country

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Year,Month,Hour,Month_Year,Income
6608,536971,21733,RED HANGING HEART T-LIGHT HOLDER,32,2010-12-03 13:40:00,2.55,12779,Poland,2010,12,13,2010-12,81.60
6609,536971,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2010-12-03 13:40:00,1.25,12779,Poland,2010,12,13,2010-12,30.00
6610,536971,37446,MINI CAKE STAND WITH HANGING CAKES,8,2010-12-03 13:40:00,1.45,12779,Poland,2010,12,13,2010-12,11.60
6611,536971,37448,CERAMIC CAKE DESIGN SPOTTED MUG,24,2010-12-03 13:40:00,1.49,12779,Poland,2010,12,13,2010-12,35.76
6612,536971,21700,BIG DOUGHNUT FRIDGE MAGNETS,24,2010-12-03 13:40:00,0.85,12779,Poland,2010,12,13,2010-12,20.40


__No. of unique orders__

In [25]:
df['InvoiceNo'].nunique()

22064

__Grouping products country-wise__

In [26]:
country_group=df[['Country','InvoiceNo']].groupby('Country').count().sort_values('InvoiceNo',ascending=True,axis=0)
country_group.head()

,InvoiceNo
Country,
Saudi Arabia,9
Bahrain,18
Czech Republic,25
Brazil,32
Lithuania,35


In [27]:
df['Description'].unique()

array(['WHITE HANGING HEART T-LIGHT HOLDER', 'WHITE METAL LANTERN',
       'CREAM CUPID HEARTS COAT HANGER', ..., 'lost',
       'CREAM HANGING HEART T-LIGHT HOLDER',
       'PAPER CRAFT , LITTLE BIRDIE'], dtype=object)

In [28]:
df['Description'].value_counts()

WHITE HANGING HEART T-LIGHT HOLDER     2327
JUMBO BAG RED RETROSPOT                2115
REGENCY CAKESTAND 3 TIER               2019
PARTY BUNTING                          1707
LUNCH BAG RED RETROSPOT                1594
                                       ... 
PINK MARSHMALLOW SCARF KNITTING KIT       1
wrongly coded-23343                       1
sold as 22467                             1
ENAMEL DINNER PLATE PANTRY                1
mouldy, unsaleable.                       1
Name: Description, Length: 4208, dtype: int64

In [29]:
df[df['Description']=='WHITE HANGING HEART T-LIGHT HOLDER'].head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Year,Month,Hour,Month_Year,Income
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,2010,12,8,2010-12,15.3
49,536373,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 09:02:00,2.55,17850,United Kingdom,2010,12,9,2010-12,15.3
66,536375,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 09:32:00,2.55,17850,United Kingdom,2010,12,9,2010-12,15.3
220,536390,85123A,WHITE HANGING HEART T-LIGHT HOLDER,64,2010-12-01 10:19:00,2.55,17511,United Kingdom,2010,12,10,2010-12,163.2
262,536394,85123A,WHITE HANGING HEART T-LIGHT HOLDER,32,2010-12-01 10:39:00,2.55,13408,United Kingdom,2010,12,10,2010-12,81.6


In [30]:
df['CustomerID'].nunique()

4340

In [31]:
df[df['CustomerID']==00000.0].head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Year,Month,Hour,Month_Year,Income


__Which customer generated how much order__

In [32]:
customer_group=df[['CustomerID','InvoiceNo']].groupby('CustomerID').count().sort_values('InvoiceNo',axis=0,ascending=False)
customer_group.head()

,InvoiceNo
CustomerID,
00000,134697
17841.0,7847
14911.0,5677
14096.0,5111
12748.0,4596


### Analyzing KPI ( Knowledge Processing Information )

We check which country generates how much sales revenue:


In [33]:
country_revenue=df[['Country','Income']].groupby('Country').sum().sort_values('Income',axis=0,ascending=False)
country_revenue.head(20)

,Income
Country,
United Kingdom,9.003098e+06
Netherlands,2.854463e+05
EIRE,2.834540e+05
Germany,2.288671e+05
France,2.097151e+05
Australia,1.385213e+05
Spain,6.157711e+04
Switzerland,5.708990e+04
Belgium,4.119634e+04


__Conclusion: The highest sales revenue is generated from UK__

Now we check out which product generates the most sales revenue:

In [34]:
product_revenue=df[['Description','Income']].groupby('Description').sum().sort_values('Income',axis=0,ascending=False)
product_revenue.head(10)

,Income
Description,
DOTCOM POSTAGE,206248.77
REGENCY CAKESTAND 3 TIER,174484.74
"PAPER CRAFT , LITTLE BIRDIE",168469.60
WHITE HANGING HEART T-LIGHT HOLDER,106292.77
PARTY BUNTING,99504.33
JUMBO BAG RED RETROSPOT,94340.05
MEDIUM CERAMIC TOP STORAGE JAR,81700.92
Manual,78112.82
POSTAGE,78101.88


#### Annual Revenue

In [35]:
annual_revenue=df[['Year','Income']].groupby('Year').sum()
annual_revenue

,Income
Year,
2010,8.237461e+05
2011,9.820814e+06


#### Monthly Revenue

In [36]:
monthly_revenue=df[['Month_Year','Income']].groupby('Month_Year').sum().sort_values('Income',axis=0,ascending=False)
monthly_revenue.head(10)

,Income
Month_Year,
2011-11,1509496.33
2011-10,1154979.30
2011-09,1058590.17
2010-12,823746.14
2011-05,770536.02
2011-06,761739.90
2011-08,737014.26
2011-07,719221.19
2011-03,717639.36


### Basket Analysis Using Apriori Algorithm

Selecting A country and analyzing it:

In [37]:
data=df[df['Country']=='Netherlands']
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Year,Month,Hour,Month_Year,Income
385,536403,22867,HAND WARMER BIRD DESIGN,96,2010-12-01 11:27:00,1.85,12791,Netherlands,2010,12,11,2010-12,177.60
386,536403,POST,POSTAGE,1,2010-12-01 11:27:00,15.00,12791,Netherlands,2010,12,11,2010-12,15.00
37952,539491,21981,PACK OF 12 WOODLAND TISSUES,12,2010-12-20 10:09:00,0.29,14646,Netherlands,2010,12,10,2010-12,3.48
37953,539491,21986,PACK OF 12 PINK POLKADOT TISSUES,12,2010-12-20 10:09:00,0.29,14646,Netherlands,2010,12,10,2010-12,3.48
37954,539491,22720,SET OF 3 CAKE TINS PANTRY DESIGN,2,2010-12-20 10:09:00,4.95,14646,Netherlands,2010,12,10,2010-12,9.90


In [54]:
#Showcasing the data in such a manner that all the column names are in one line and finding which invoice number ordered 
#how much amount

basket=data.groupby(['InvoiceNo','Description'])['Quantity'].sum().unstack().fillna(0)
basket

Description,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,I LOVE LONDON MINI RUCKSACK,SPACEBOY BABY GIFT SET,10 COLOUR SPACEBOY PEN,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,16 PIECE CUTLERY SET PANTRY DESIGN,...,WRAP RED APPLES,WRAP WEDDING DAY,YELLOW METAL CHICKEN HEART,ZINC HEART T-LIGHT HOLDER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC WILLIE WINKIE CANDLE STICK
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536403,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
539491,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
539731,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,192.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
541206,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
541570,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
578143,0.0,0.0,0.0,0.0,0.0,384.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
579528,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
581175,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Next we modify the table in such a way that we can identify whether the product is present or not (This can also be checked in the above function - 0 means product not there and some value is there that means product was there but the below method is the better method to check it) 

0 - product not present<br>1 - product present

In [55]:
basket_model=basket.applymap(lambda x:1 if x>0 else 0)
basket_model

Description,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,I LOVE LONDON MINI RUCKSACK,SPACEBOY BABY GIFT SET,10 COLOUR SPACEBOY PEN,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,16 PIECE CUTLERY SET PANTRY DESIGN,...,WRAP RED APPLES,WRAP WEDDING DAY,YELLOW METAL CHICKEN HEART,ZINC HEART T-LIGHT HOLDER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC WILLIE WINKIE CANDLE STICK
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536403,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
539491,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
539731,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
541206,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
541570,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
578143,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
579528,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
581175,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


__We are setting the minimum support to 0.08:__

In [57]:
popular_sets=apriori(basket_model,min_support=0.08,use_colnames=True)
popular_sets

#use_colnames is set to True becsuse i want to display the name of the columns in the itemsets !! By default apriori
#returns the dataset in which the indices of itemset are displayed in place of their names hence use_colnames has been set
#to True

,support,itemsets
0,0.105263,(72 SWEETHEART FAIRY CAKE CASES)
1,0.094737,(CARD BIRTHDAY COWBOY)
2,0.126316,(CARD DOLLY GIRL )
3,0.094737,(CARD GINGHAM ROSE )
4,0.084211,(CHARLOTTE BAG APPLES DESIGN)
...,...,...
199,0.084211,"(DOLLY GIRL LUNCH BOX, ROUND SNACK BOXES SET O..."
200,0.084211,"(DOLLY GIRL LUNCH BOX, ROUND SNACK BOXES SET O..."
201,0.084211,"(DOLLY GIRL LUNCH BOX, WOODLAND CHARLOTTE BAG,..."
202,0.084211,"(DOLLY GIRL LUNCH BOX, WOODLAND CHARLOTTE BAG,..."


In [59]:
rules=association_rules(popular_sets,metric='lift',min_threshold=1)
rules.head()

#min_threshold is the minimum value of metric that we want to set as threshold . That means we only want those data here in
#which the value of lift (beacause here our metric is lift) is greater than 1.

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(DOLLY GIRL LUNCH BOX),(72 SWEETHEART FAIRY CAKE CASES),0.231579,0.105263,0.094737,0.409091,3.886364,0.070360,1.514170
1,(72 SWEETHEART FAIRY CAKE CASES),(DOLLY GIRL LUNCH BOX),0.105263,0.231579,0.094737,0.900000,3.886364,0.070360,7.684211
2,(SPACEBOY LUNCH BOX ),(72 SWEETHEART FAIRY CAKE CASES),0.294737,0.105263,0.094737,0.321429,3.053571,0.063712,1.318560
3,(72 SWEETHEART FAIRY CAKE CASES),(SPACEBOY LUNCH BOX ),0.105263,0.294737,0.094737,0.900000,3.053571,0.063712,7.052632
4,(SPACEBOY BIRTHDAY CARD),(CARD BIRTHDAY COWBOY),0.178947,0.094737,0.084211,0.470588,4.967320,0.067258,1.709942


From the above data we can clearly observe that the lift values are pretty high !! This means there are really strong correlations in this data . Lets check the data for much higher levels of lift and confidence !!

In [61]:
rules[(rules['lift']>=5) & (rules['confidence']>=1)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
109,(PACK OF 60 MUSHROOM CAKE CASES),(SPACEBOY BIRTHDAY CARD),0.084211,0.178947,0.084211,1.0,5.588235,0.069141,inf
331,"(SPACEBOY BIRTHDAY CARD, LUNCH BAG RED RETROSPOT)",(PLASTERS IN TIN SPACEBOY),0.084211,0.126316,0.084211,1.0,7.916667,0.073573,inf
332,"(PLASTERS IN TIN SPACEBOY, LUNCH BAG RED RETRO...",(SPACEBOY BIRTHDAY CARD),0.084211,0.178947,0.084211,1.0,5.588235,0.069141,inf
472,"(DOLLY GIRL LUNCH BOX, PLASTERS IN TIN SPACEBOY)","(ROUND SNACK BOXES SET OF4 WOODLAND , SPACEBOY...",0.084211,0.189474,0.084211,1.0,5.277778,0.068255,inf
488,"(DOLLY GIRL LUNCH BOX, RED RETROSPOT CHARLOTTE...","(ROUND SNACK BOXES SET OF4 WOODLAND , SPACEBOY...",0.094737,0.189474,0.094737,1.0,5.277778,0.076787,inf
491,"(SPACEBOY LUNCH BOX , RED RETROSPOT CHARLOTTE ...","(DOLLY GIRL LUNCH BOX, ROUND SNACK BOXES SET O...",0.094737,0.168421,0.094737,1.0,5.937500,0.078781,inf
516,"(DOLLY GIRL LUNCH BOX, SPACEBOY BIRTHDAY CARD)","(ROUND SNACK BOXES SET OF4 WOODLAND , SPACEBOY...",0.084211,0.189474,0.084211,1.0,5.277778,0.068255,inf
568,"(SPACEBOY BIRTHDAY CARD, ROUND SNACK BOXES SET...",(PLASTERS IN TIN SPACEBOY),0.084211,0.126316,0.084211,1.0,7.916667,0.073573,inf
569,"(PLASTERS IN TIN SPACEBOY, ROUND SNACK BOXES S...",(SPACEBOY BIRTHDAY CARD),0.084211,0.178947,0.084211,1.0,5.588235,0.069141,inf
572,"(SPACEBOY BIRTHDAY CARD, LUNCH BAG RED RETROSPOT)","(PLASTERS IN TIN SPACEBOY, ROUND SNACK BOXES S...",0.084211,0.126316,0.084211,1.0,7.916667,0.073573,inf


From the above data we can see that for example if a person buys PACK OF 60 MUSHROOM CAKE CASES then there is high chance he will buy SPACEBOY BIRTHDAY CARD . Similarly if a person buys SPACEBOY BIRTHDAY CARD and LUNCH BAG RED RETROSPOT there is a high chance he will buy PLASTERS IN TIN SPACEBOY .

### Recommendations

In [62]:
rules['antecedents'][109]

frozenset({'PACK OF 60 MUSHROOM CAKE CASES'})

In [63]:
basket_model['PACK OF 60 MUSHROOM CAKE CASES'].sum()

8

In [64]:
basket_model['SPACEBOY BIRTHDAY CARD'].sum()

17

### At this point we can see how great the possibilities are to use the popularity of one product to increase the sales of another product . For example we see the value of lift is pretty high for "PACK OF 60 MUSHROOM CAKE CASES' and "SPACEBOY BIRTHDAY CARD" , this means that whenever 'PACK OF 60 MUSHROOM CAKE CASES' is bought 'SPACEBOY BIRTHDAY CARD' is also bought/chances are there it is bought . But if we observe the above 2 cells - 'SPACEBOY BIRTHDAY CARD' is bought 17 times whereas 'PACK OF 60 MUSHROOM CAKE CASES' is bought 8 times only !! Therefore if we club both the products together then the sales of 'PACK OF 60 MUSHROOM CAKE CASES' will be benefitted due to the sales of 'SPACEBOY BIRTHDAY CARD'